<font size=4><b>Created by yunsuxiaozi</b></font>

<font size=5><b>Linking Writing Processes to Writing Quality</b></font>


## Import data and necessary libraries

In [ ]:
#https://www.kaggle.com/code/cody11null/lgbm-x2-nn#LightAutoML-NN-(DenseLight)-prediction

#安装包,--no-index 表示不从 PyPI（Python Package Index）上下载安装包
#-U是upgrade -q是减少输出信息,--find-links=是指定链接 后面是安装的版本.
!pip install --no-index -U -q --find-links=/kaggle/input/lightautoml-038-dependecies lightautoml==0.3.8
!pip install --no-index -U -q --find-links=/kaggle/input/lightautoml-038-dependecies pandas==2.0.3

In [ ]:
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
import gc#垃圾回收模块

import re#用于正则表达式提取
from collections import Counter#用于对一组元素计数

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML#自动化表格机器学习模型的库
from lightautoml.tasks import Task#定义机器学习任务的库

#设置随机种子,保证模型可以复现
import random
np.random.seed(2023)
random.seed(2023)
#import optuna#自动超参数优化软件框架
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

In [ ]:
train_logs=pd.read_csv("/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv")
print(f"len(train_logs):{len(train_logs)}")
train_logs.head()

In [ ]:
train_scores=pd.read_csv("/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv")
print(f"len(train_scores):{len(train_scores)}")
train_scores.head()

In [ ]:
train_df=pd.merge(train_logs,train_scores,on="id",how="left")
print(f"len(train_df):{len(train_df)}")
train_df.head()

In [ ]:
#统计‘q’ '.' ‘ ’的一个函数.
def getEssays(df):
    #获取传入的df的这几列.
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']]
    #取出activity不等于'Nonproduction'的那些数据
    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']
    #统计每个id的出现次数,不排序
    valCountsArr = textInputDf['id'].value_counts(sort=False).values
    #最后的下标
    lastIndex = 0
    #创建一个新的序列对象.
    essaySeries = pd.Series()
    #index是第几个id,valCount是出现次数
    for index, valCount in enumerate(valCountsArr):
        #取出第i个id的['activity', 'cursor_position', 'text_change']
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]
        #跳到下一个id的index
        lastIndex += valCount
        essayText = ""
        for Input in currTextInput.values:
            #input[0]是这个id的activity
            if Input[0] == 'Replace':
                #text_change按照' => '分开 replaceTxt:[' qqq qqqqq ', ' ']
                replaceTxt = Input[2].split(' => ')#应该是A=>B的操作
                #input[1]是鼠标位置,是一个数字 鼠标位置-len()
                #这是一个字符串的转换操作,由replaceTxt[0]转成replaceTxt[1] 
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] +essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue
            if Input[0] == 'Paste':#粘贴
                #print(f"input[2]:{Input[2]}") #input[2]:qqqqqqqqqqq 
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue
            if Input[0] == 'Remove/Cut':#删除剪切 在Input[1]的位置删除Input[2]
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue
            #如果是Move from
            if "M" in Input[0]:
                #[284, 292] To [282, 290] 把[284, 292]这8行移动到[282,290]
                croppedTxt = Input[0][10:]
                #from和to的4个数字分开.
                splitTxt = croppedTxt.split(' To ')
                valueArr = [item.split(', ') for item in splitTxt]
                moveData = (int(valueArr[0][0][1:]), 
                            int(valueArr[0][1][:-1]), 
                            int(valueArr[1][0][1:]), 
                            int(valueArr[1][1][:-1]))
                #行号不相等,如果相等,等于什么都没有做
                if moveData[0] != moveData[2]:
                    #行号小于 
                    if moveData[0] < moveData[2]:
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] +\
                        essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    #行号大于
                    else:
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] +\
                        essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
            #相当于是个check    
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        #对应id对应论文
        essaySeries[index] = essayText
    #id
    essaySeries.index =  textInputDf['id'].unique()
    return pd.DataFrame(essaySeries, columns=['essay'])


train_essays = pd.read_csv('../input/writing-quality-challenge-constructed-essays/train_essays_02.csv')
train_essays.index = train_essays["Unnamed: 0"]
train_essays.index.name = None
train_essays.drop(columns=["Unnamed: 0"], inplace=True)
print(f"len(train_essays):{len(train_essays)}")
train_essays['len_essay']=train_essays['essay'].apply(len)
train_essays.head()

## Feature engineer

In [ ]:
#获取数据中第25%的数值
def q1(x):
    return x.quantile(0.25)
#获取数据中第75%的数值
def q3(x):
    return x.quantile(0.75)
#对数据聚合的操作:数量,均值,方差,最小值,最大值,第一,最后,均值标准误差,25%,50%,75%,偏斜度,峰度,求和
AGGREGATIONS = ['count', 'mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']

#将传入的论文df转成句子
def split_essays_into_sentences(df):
    essay_df = df#传入的df就是论文的df
    essay_df['id'] = essay_df.index#index就是每个人的编号
    #对句子按照. ? !进行拆分. 得到一个拆分后的列表.
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    # essay1 [1,2,3] essay2[4,5] ->5行 essay1 1  // essay1 2 // essay1 3 // essay2 1 // essay2 2
    essay_df = essay_df.explode('sent')
    #将换行符'\n'变成空白字符 strip 去除行头和行尾的空白字符.
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
    #统计一下每个句子的长度 
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    #求一下每个句子单词的个数.
    essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
    #去掉那些句子长度为0的数据
    essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
    return essay_df

#对句子的一些特征进行统计.
def compute_sentence_aggregations(df):
    #对每个id的句子长度和每个句子的单词个数进行统计学变量的统计.并且拼接在一起.
    sent_agg_df = pd.concat(
        [df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    )
    #sent_agg_df.columns: (       'sent_len',  'count')-> 'sent_len_count'
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]#列名变换
    sent_agg_df['id'] = sent_agg_df.index#将索引保存
    sent_agg_df = sent_agg_df.reset_index(drop=True)#重置索引,删除index
    #一句话里词的个数的count,其实就是有多少句话,也就是sent_len的count.重复了,故去掉.
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    #sent_len_count其实就是有多少句话,故rename.
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df

#将论文根据换行符划分为段落.(每段有多少句话为什么没有统计?)
def split_essays_into_paragraphs(df):
    essay_df = df
    essay_df['id'] = essay_df.index
    #按照'\n'划分成段落 [1,2,3]
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
    #[论文1 [段落1 段落2,……]->[论文1 段落1 // 论文1 段落2]
    essay_df = essay_df.explode('paragraph')
    #统计段落的长度
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    #统计每个段落的词数
    essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
    #将段落长度为0的数据去掉.
    essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
    return essay_df

#对段落的长度和词数用统计学变量,和上面句子的代码一致.
def compute_paragraph_aggregations(df):
    paragraph_agg_df = pd.concat(
        [df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    ) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df
#训练数据论文划分为句子
train_sent_df = split_essays_into_sentences(train_essays)
#句子特征提取
train_sent_agg_df = compute_sentence_aggregations(train_sent_df)
#论文划分成段落
train_paragraph_df = split_essays_into_paragraphs(train_essays)
#段落特征提取
train_paragraph_agg_df = compute_paragraph_aggregations(train_paragraph_df)

In [ ]:
test_logs=pd.read_csv("/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv")
print(f"len(test_logs):{len(test_logs)}")
#测试数据得到论文,然后得到句子和段落的特征
test_essays = getEssays(test_logs)
test_essays['len_essay']=test_essays['essay'].apply(len)
test_sent_agg_df = compute_sentence_aggregations(split_essays_into_sentences(test_essays))
test_paragraph_agg_df = compute_paragraph_aggregations(split_essays_into_paragraphs(test_essays))
test_logs.head()

In [ ]:
from collections import defaultdict#一个存在默认值的字典,访问不存在的值时抛出的是默认值

class Preprocessor:#数据预处理的一个类
    
    def __init__(self, seed):
        self.seed = seed#随机种子
        
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste','Move From']#这是activity的一列
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']#down_event中选出一些重要的
        self.text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']#text_change中选出一些重要的
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']#down_event中的一些标点符号
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 100]#滞后项
        
        #这里是用于存储每个activity的idf值
        self.idf = defaultdict(float)#创建了一个float类型的字典,如果访问不存在,默认值为0.0
    #将列表转成字符串
    def list_to_str(self,lst):
        return ','.join(lst)#按照','来进行拼接
    def activity_each_word_speed(self,df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        tmp_df['activity']=tmp_df['activity'].apply(self.list_to_str)
        #将逗号换成空格
        tmp_df['activity']=tmp_df['activity'].apply(lambda x: x.replace(",",' '))

        tmp_df['activity']=tmp_df['activity'].apply(lambda x: re.split('space',x))

        tmp_df=tmp_df.explode('activity')

        tmp_df['each_word_speed'] = tmp_df['activity'].apply(lambda x: len(x))

        return tmp_df['each_word_speed'].groupby(tmp_df['id']).agg(AGGREGATIONS).reset_index().drop(['id'],axis=1,inplace=True)
        
    #统计df对象中activity的count
    def activity_counts(self, df):
        #对每个id的所有activity组合成一个列表
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        #创建一个空列表
        ret = list()
        for li in tmp_df['activity'].values:#取出一个人的activity列表
            items = list(Counter(li).items())#转成[(activity1:count1),(activity2:count2),……]
            di = dict()#一个空字典
            #每个activity初始化为0
            for k in self.activities:
                di[k] = 0
            #统计每个activity的count
            for item in items:
                k, v = item[0], item[1]#k:activity v:count
                if k in di:
                    di[k] = v
            #加上这个人的每个activity的count
            ret.append(di)
        #转成pandas类型
        ret = pd.DataFrame(ret)
        #给表格的每列换个名字
        cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        #每列元素求和,文章中出现的总次数
        cnts = ret.sum(1)

        #前面是词袋模型,这里转成tf-idf模型
        for col in cols:#activity_i_count
            if col in self.idf.keys():#字典里如果已经有这个key了
                idf = self.idf[col]
            else:#不在这个字典里
                #计算idf=log(数据量/(某列和+1))
                idf = np.log(df.shape[0] / (ret[col].sum() + 1))
                self.idf[col] = idf#将col的idf加入字典
            #ret[col] / cnts :给定文章的次数/在文章中出现的总次数,为什么取log再加1不知道
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret#tf-idf

    #这个是event的tf-idf模型,这里可能有down_event和up_event,故colname单独设置
    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tmp_df[colname].values:
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret

    #text_change的tf-idf模型
    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tmp_df['text_change'].values:
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf
            
        return ret
    #统计标点之类的出现的次数,不过这次是直接将它们相加做统计的.(可能这样比tf-idf好?)
    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tmp_df['down_event'].values:
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:#只要在这张表里,就相加
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret


    def get_input_words(self, df):
        #~是取反的布尔值 取出text_change 中不包含 => 且不是Nochange的
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        #在drop掉包含 => 和Nochange之后 按id打包成列表
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        #将列表连接成一个整体
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        #用正则表达式子匹配一个或者多个'q'字符
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        #统计len,也就是统计text_change中有多少个有q的字符
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        #求均值,方差,最大值,取到np.nan就设置为0
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    #对df做特征工程
    def make_feats(self, df):
        
        print("Starting to engineer features")
        #创建一个只有id一列的表格
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        #做时序上的特征工程
        print("Engineering time data")
        for gap in self.gaps:
            print(f"-> for gap {gap}")
            #利用up_time的shift创造action_time_gap
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)

        #对cursor_position做特征工程,这个就是自己-自己
        print("Engineering cursor position data")
        for gap in self.gaps:
            print(f"-> for gap {gap}")
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            #取了绝对值,鼠标向前移动也是移动了.
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)

        #对word_count做类似的特征工程,词数减少也是移动了.
        print("Engineering word count data")
        for gap in self.gaps:
            print(f"-> for gap {gap}")
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)
        
        print("Engineering statistical summaries for features")
        #需要对哪些特征做哪些统计变量,这些都是大佬统计好的,就不做修改了.
        feats_stat = [
            ('event_id', ['max']),
            ('up_time', ['max']),
            ('action_time', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ('cursor_position', ['nunique', 'max', 'quantile', 'sem', 'mean']),
            ('word_count', ['nunique', 'max', 'quantile', 'sem', 'mean'])]
        #滞后特征的统计变量用for循环进行添加
        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'cursor_position_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt])
            ])
        
        pbar = feats_stat
        for item in pbar:
            colname, methods = item[0], item[1]#取出某列特征和需要进行的统计学的量'max'
            for method in methods:
                #转成能放入agg的方法
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                #添加到feats里.
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left')

        #调用方法求activity的tf-idf
        print("Engineering activity counts data")
        tmp_df = self.activity_each_word_speed(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        #调用方法求down_event和up_event的tf-idf
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        # input words
        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')

        # compare feats
        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
        feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_sum'] / feats['up_time_max']
        
        print("Done!")
        return feats
preprocessor = Preprocessor(seed=2023)
print("Engineering features for training data")

train_feats = preprocessor.make_feats(train_logs)
print("-"*25)
print("Engineering features for test data")
test_feats = preprocessor.make_feats(test_logs)
#将训练数据中有缺失值的列作统计
nan_cols = train_feats.columns[train_feats.isna().any()].tolist()
print(f"len(nan_cols):{len(nan_cols)},nan_cols:{nan_cols}")
#将训练数据和测试数据去掉nan的列.
train_feats = train_feats.drop(columns=nan_cols)
test_feats = test_feats.drop(columns=nan_cols)

keys=train_feats.keys().values
unique_cols=[key for key in keys if train_feats[key].nunique()<2]
print(f"len(unique_cols):{len(unique_cols)},unique_cols:{unique_cols}")
#将训练数据和测试数据去掉unique的列.
train_feats = train_feats.drop(columns=unique_cols)
test_feats = test_feats.drop(columns=unique_cols)

In [ ]:
#根据id对某些特征求了一些统计学变量
train_agg_fe_df = train_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
#列名转成:"特征_统计学量"
train_agg_fe_df.columns = ['_'.join(x) for x in train_agg_fe_df.columns]
#给列名添加上前缀tmp_
train_agg_fe_df = train_agg_fe_df.add_prefix("tmp_")
#将行号设置为[0,1,2,3,……]
train_agg_fe_df.reset_index(inplace=True)

#对测试集做同样的操作
test_agg_fe_df = test_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
test_agg_fe_df.columns = ['_'.join(x) for x in test_agg_fe_df.columns]
test_agg_fe_df = test_agg_fe_df.add_prefix("tmp_")
test_agg_fe_df.reset_index(inplace=True)

#将文件合并起来
train_feats = train_feats.merge(train_agg_fe_df, on='id', how='left')
test_feats = test_feats.merge(test_agg_fe_df, on='id', how='left')

In [ ]:
data = []

for logs in [train_logs, test_logs]:
    #up_time向后移动并且用down_time填充缺失的位置
    logs['up_time_lagged'] = logs.groupby('id')['up_time'].shift(1).fillna(logs['down_time'])
    #(down_time减上一个时刻的up_time) /1000是单位转换
    logs['time_diff'] = abs(logs['down_time'] - logs['up_time_lagged']) / 1000

    group = logs.groupby('id')['time_diff']
    #延迟时间的max,min,median
    largest_lantency = group.max()
    smallest_lantency = group.min()
    median_lantency = group.median()
    #down_time的first /1000是做单位转换吧
    initial_pause = logs.groupby('id')['down_time'].first() / 1000
    #分层次求和
    pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x <= 1)).sum())
    pauses_1_sec = group.apply(lambda x: ((x > 1) & (x <= 1.5)).sum())
    pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x <= 2)).sum())
    pauses_2_sec = group.apply(lambda x: ((x > 2) & (x <= 3)).sum())
    pauses_3_sec = group.apply(lambda x: (x > 3).sum())

    data.append(pd.DataFrame({
        'id': logs['id'].unique(),
        #延迟
        'largest_lantency': largest_lantency,
        'smallest_lantency': smallest_lantency,
        'median_lantency': median_lantency,
        'initial_pause': initial_pause,
        'pauses_half_sec': pauses_half_sec,
        'pauses_1_sec': pauses_1_sec,
        'pauses_1_half_sec': pauses_1_half_sec,
        'pauses_2_sec': pauses_2_sec,
        'pauses_3_sec': pauses_3_sec,
    }).reset_index(drop=True))

train_eD592674, test_eD592674 = data

gc.collect()

#将延时的特征加入
train_feats = train_feats.merge(train_eD592674, on='id', how='left')
test_feats = test_feats.merge(test_eD592674, on='id', how='left')
#在训练的feature中加入标签
train_feats = train_feats.merge(train_scores, on='id', how='left')

#加上句子特征和段落特征.
train_feats = train_feats.merge(train_sent_agg_df, on='id', how='left')
train_feats = train_feats.merge(train_paragraph_agg_df, on='id', how='left')
train_feats['len_essay']=train_essays['len_essay'].values
test_feats = test_feats.merge(test_sent_agg_df, on='id', how='left')
test_feats = test_feats.merge(test_paragraph_agg_df, on='id', how='left')
test_feats['len_essay']=test_essays['len_essay'].values
#id这列就去掉了
train_feats.drop(['id'],axis=1,inplace=True)
test_feats.drop(['id'],axis=1,inplace=True)
train_feats.fillna(0,inplace=True)
test_feats.fillna(0,inplace=True)

In [ ]:
very_low_score=train_feats[train_feats['score']==0.5]
low_score=train_feats[train_feats['score']==1.0]
high_score=train_feats[train_feats['score']==5.5]
very_high_score=train_feats[train_feats['score']==6]
print(f"very_low_score['len_essay'].values:{very_low_score['len_essay'].values}")
print(f"low_score['len_essay'].values:{low_score['len_essay'].values}")
print(f"very_high_score['len_essay'].values:{very_high_score['len_essay'].values}")
print(f"high_score['len_essay'].values:{high_score['len_essay'].values}")

## lightAutoml

In [ ]:
N_THREADS = 4
N_FOLDS = 5
TEST_SIZE = 0.2
TIMEOUT = 5 * 3600
TARGET_NAME='score'
#评估指标是RMSE
def RMSE(y_true,y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))
#任务是回归任务,损失是MSE,评估指标是RMSE
task = Task('reg', loss = 'mse', metric = RMSE)
roles = {
    'target': TARGET_NAME
}
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': 2023, 'advanced_roles': False}
)
#%%time 
oof_pred = automl.fit_predict(train_feats, roles = roles, verbose = 3)
print(automl.create_model_str_desc())
print(f'TRAIN out-of-fold score: {RMSE(train_feats[TARGET_NAME].values, oof_pred.data[:, 0])}')

## Pseudo label

In [ ]:
test_pred=automl.predict(test_feats).data.T[0]
test_feats[TARGET_NAME] =test_pred
total_feats=pd.concat((train_feats,test_feats),axis=0)

#这是以前的参数. https://www.kaggle.com/code/yunsuxiaozi/crazy-feature-engineer-make-me-crazy
lgbm_params1={'random_state': 1730, 'n_estimators': 228,#428, 
              'reg_alpha': 0.09004045282744186, 'reg_lambda': 4.173656114633056, 
              'colsample_bytree': 0.8713036303176558, 'subsample': 0.8184951070958928, 
              'learning_rate': 0.08775936917066098, 'num_leaves': 8, 'min_child_samples': 100}

lgbm_params2= {'random_state': 1094, 'n_estimators': 189,#489, 
              'reg_alpha': 0.0038009385912019253, 'reg_lambda': 2.440515046794778,
              'colsample_bytree': 0.5623454637024369, 'subsample': 0.7745734180496221, 
              'learning_rate': 0.15375733522524826, 'num_leaves': 13, 
              'min_child_samples': 87}



from sklearn.model_selection import KFold#在机器学习库中导入k折交叉验证的函数
from lightgbm import  LGBMRegressor 

def RMSE(y_true,y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))
print("start fit.")
folds = 10#将数据分成10份
y=total_feats['score']
X=total_feats.drop(['score'],axis=1)

train_RMSE=[]
valid_RMSE=[]
# 存储已学习模型的列表
models = []

#将数据集随机打乱,并分成folds份
kf = KFold(n_splits=folds, shuffle=True, random_state=2023) 

#从x_train中按照9:1的比例分成训练集和验证集,并取出下标
for train_index, valid_index in kf.split(X):
    
    #根据下标取出训练集和验证集的数据
    x_train_cv = X.iloc[train_index]
    y_train_cv = y.iloc[train_index]
    x_valid_cv =X.iloc[valid_index]
    y_valid_cv = y.iloc[valid_index]
    
    model = LGBMRegressor(**lgbm_params1)
    
    #模型用x_train_cv去训练,用x_train_cv和x_valid_cv一起去评估
    model.fit(
        x_train_cv,y_train_cv,
        eval_set = [(x_train_cv, y_train_cv), (x_valid_cv, y_valid_cv)], 
        early_stopping_rounds=100,
        verbose = 100, #迭代100次输出一个结果
    )
    
    #对训练集进行预测
    y_pred_train = model.predict(x_train_cv)        
    #对验证集进行预测
    y_pred_valid = model.predict(x_valid_cv) 
    
        
    y_pred_train=np.where(y_pred_train<=0.5,0.5,y_pred_train)
    y_pred_train=np.where(y_pred_train>=6,6,y_pred_train)    
    
    y_pred_valid=np.where(y_pred_valid<=0.5,0.5,y_pred_valid)
    y_pred_valid=np.where(y_pred_valid>=6,6,y_pred_valid)
    
    train_rmse=RMSE(y_pred_train,y_train_cv)
    valid_rmse=RMSE(y_pred_valid,y_valid_cv)

    
    train_RMSE.append(train_rmse)
    valid_RMSE.append(valid_rmse)
    
    #将model保存进列表中
    models.append(model)
    
    model = LGBMRegressor(**lgbm_params2)
    
    #模型用x_train_cv去训练,用x_train_cv和x_valid_cv一起去评估
    model.fit(
        x_train_cv,y_train_cv,
        eval_set = [(x_train_cv, y_train_cv), (x_valid_cv, y_valid_cv)], 
        early_stopping_rounds=100,
        verbose = 100, #迭代100次输出一个结果
    )
    
    #对训练集进行预测
    y_pred_train = model.predict(x_train_cv)        
    #对验证集进行预测
    y_pred_valid = model.predict(x_valid_cv) 
    
        
    y_pred_train=np.where(y_pred_train<=0.5,0.5,y_pred_train)
    y_pred_train=np.where(y_pred_train>=6,6,y_pred_train)    
    
    y_pred_valid=np.where(y_pred_valid<=0.5,0.5,y_pred_valid)
    y_pred_valid=np.where(y_pred_valid>=6,6,y_pred_valid)
    
    train_rmse=RMSE(y_pred_train,y_train_cv)
    valid_rmse=RMSE(y_pred_valid,y_valid_cv)

    
    train_RMSE.append(train_rmse)
    valid_RMSE.append(valid_rmse)
    
    #将model保存进列表中
    models.append(model)
    
    print(f"train_RMSE:{train_RMSE},valid_RMSE:{valid_RMSE}")

train_RMSE=np.array(train_RMSE)
valid_RMSE=np.array(valid_RMSE)

print(f"mean_train_RMSE:{np.mean(train_RMSE)}")
print(f"mean_valid_RMSE:{np.mean(valid_RMSE)}")

## Submission

In [ ]:
test_X=test_feats.drop(['score'],axis=1).values
#用每个保存的模型都对x_test预测一次,然后取平均值
preds_test = []

for model in models:
    
    pred = model.predict(test_X)
    
    preds_test.append(pred)
    
#将预测结果转成np.array
preds_test_np = np.array(preds_test)
#按列对每个模型的预测结果取平均值
test_pred=np.mean(preds_test_np,axis=0)
test_pred

In [ ]:
submission=pd.read_csv("/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv")
submission[TARGET_NAME] =test_pred
submission.to_csv('submission.csv', index = False)
submission.head()